In [ ]:
%run "0_Workspace_setup.ipynb"


from NHM_helpers.NHM_helpers import hrus_by_poi, hrus_by_seg, subset_stream_network
from NHM_helpers.map_template import *
from NHM_helpers.NHM_Assist_utilities import make_plots_par_vals

In [ ]:
# Create dataframes for hydrofabric elements
hru_gdf, hru_cal_level_txt = create_hru_gdf(
    NHM_dir,
    model_dir,
    GIS_format,
    param_filename,
    nhru_params,
    nhru_nmonths_params,
)

seg_gdf = create_segment_gdf(
    model_dir,
    GIS_format,
    param_filename,
)

nwis_gages_aoi = fetch_nwis_gage_info(
    model_dir,
    control_file_name,
    nwis_gage_nobs_min,
    hru_gdf,
)

poi_df = create_poi_df(
    model_dir,
    param_filename,
    control_file_name,
    hru_gdf,
    nwis_gages_aoi,
    gages_file,
)

default_gages_file = create_default_gages_file(
    model_dir,
    nwis_gages_aoi,
    poi_df,
)

gages_df = read_gages_file(
    model_dir,
    poi_df,
    nwis_gages_file,
    gages_file,
)

HW_basins_gdf, HW_basins = make_HW_cal_level_files(hru_gdf)

con.print(
    f"{workspace_txt}",
    f"\n{hru_cal_level_txt}",
)

## Build plots at each POI that show par values of contributing HRUs

In [ ]:
make_plots_par_vals(
    poi_df,
    hru_gdf,
    param_filename,
    nhru_params,
    nhru_nmonths_params,
    Folium_maps_dir,
)

## Now select a parameter to view on the map

In [ ]:
from ipywidgets import widgets

cal_hru_params = nhru_params + nhru_nmonths_params

par_sel = cal_hru_params[4]
# sel_flag = False

v = widgets.Dropdown(
    options=cal_hru_params,
    value=cal_hru_params[0],
    description="Select a parameter to view in plots:",
)


def on_change(change):
    global par_sel, sel_flag
    if change["type"] == "change" and change["name"] == "value":
        par_sel = v.value
        # sel_flag = True


v.observe(on_change)

display(v)

### After selecting from the dropdown above, click here, then go to Run in the Menu

In [ ]:
mo_num_dict = {
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12,
}

mo_names = list(mo_num_dict.keys())

mo_name = "July"  # set default value
mo_sel = mo_num_dict[mo_name]

try:
    pdb.get(par_sel).dimensions["nmonths"].size

except:
    print(f"{par_sel} dimensioned by nhru only.")
    mo_sel = None

else:
    print(f"{par_sel} dimensioned by nhru and nmonths. Please select a month below.")
    # mo_name = "July"  # set default value
    # mo_sel = mo_num_dict[mo_name]

    m = widgets.Dropdown(
        options=mo_names,
        value=mo_names[6],  # set default value
        description="Select a month to display on the map:",
    )

    def on_change(change):
        global mo_sel, mo_name, mo_num_dict
        if change["type"] == "change" and change["name"] == "value":
            mo_name = m.value
            mo_sel = mo_num_dict[mo_name]

    m.observe(on_change)

    display(m)

### After selecting from the dropdown above, click here, then go to Run in the Menu

## Map selected parameter value for HRUs

### Create map

In [ ]:
from IPython.display import display
from folium.plugins import MeasureControl
from folium.utilities import Element
from folium.plugins import FloatImage
import base64
import webbrowser

m3 = folium.Map()

pfile_lat, pfile_lon, zoom, cluster_zoom = folium_map_elements(hru_gdf, poi_df, "")
USGSHydroCached_layer, USGStopo_layer, Esri_WorldImagery, OpenTopoMap = (
    folium_map_tiles()
)
minimap = create_minimap()


m3 = folium.Map(
    location=[pfile_lat, pfile_lon],
    # width=1000, height=600,
    tiles=USGSHydroCached_layer,
    zoom_start=zoom,
    control_scale=True,
)

USGStopo_layer.add_to(m3)
OpenTopoMap.add_to(m3)
Esri_WorldImagery.add_to(m3)

# Add widgets
m3.add_child(minimap)
m3.add_child(MeasureControl(position="bottomright"))

hru_map, val_bar_image, value_min, value_max, same_value, color_bar = (
    create_nhru_par_map(
        param_filename,
        hru_gdf,
        par_sel,
        mo_sel,
        mo_name,
        nhru_params,
        Folium_maps_dir,
    )
)
# fig.show()
marker_cluster_label_hru = create_hru_label(hru_gdf, cluster_zoom)
marker_cluster, marker_cluster_label_poi = create_poi_paramplot_marker_cluster(
    poi_df,
    hru_gdf,
    Folium_maps_dir,
    cluster_zoom,
    par_sel,
)

hru_map.add_to(m3)

hw_basins_map = folium.GeoJson(
    HW_basins, style_function=hw_basin_style, name="HW basin boundary"
).add_to(m3)

marker_cluster_label_hru.add_to(m3)

seg_map = create_segment_map_show(seg_gdf)
seg_map.add_to(m3)
marker_cluster.add_to(m3)
marker_cluster_label_poi.add_to(m3)


plugins.Fullscreen(position="topleft").add_to(m3)
folium.LayerControl(collapsed=True, position="bottomright", autoZIndex=True).add_to(m3)

if not color_bar:
    scale_bar_txt = f"All {par_sel} values are {same_value} in the model domain. No value scale bar rendered."
    fig = None  # fig, ax = plt.subplots(figsize=(18, 0.5))
else:
    scale_bar_txt = f"Values for {par_sel} range from {value_min:.2f} to {value_max:.2f} {pdb.get(par_sel).meta['units']} in the model domain."


if mo_sel is None:
    mo_txt = " "
    map_file = f"{html_maps_dir}/{par_sel}_map.html"
else:
    mo_txt = f"{mo_name} "
    map_file = f"{html_maps_dir}/{par_sel}_{mo_name}_map.html"


title_html = f"<h1 style='position:absolute;z-index:100000;font-size: 28px;left:26vw;text-shadow: 3px  3px  3px white,-3px -3px  3px white,3px -3px  3px white,-3px  3px  3px white; '><strong>NHM {mo_txt}{par_sel}</strong><br><h1 style='position:absolute;z-index:100000;font-size: 20px;left:31vw;right:5vw; top:4vw;text-shadow: 3px  3px  3px white,-3px -3px  3px white,3px -3px  3px white,-3px  3px  3px white; '> {pdb.get(par_sel).meta['help']}. {scale_bar_txt}</h1>"
# title_html = f"<h1 align='center' style='font-size: 14px;'><strong>NHM simulated {sel_year} {output_var_sel}, {var_units}</strong><br><h1 align='center' style='font-size: 14px;'> {var_desc}. {scale_bar_txt}</h1>"
m3.get_root().html.add_child(Element(title_html))

m3.add_child(val_bar_image)

m3.save(map_file)

webbrowser.open(map_file)